In [5]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import GridSearchCV
import os
import json
from pathlib import Path

class CNFDatasetExperiment:
    """
    Experiment runner for Decision Tree classification on CNF datasets.
    """
    
    def __init__(self, data_dir='./all_data'):
        """
        Initialize the experiment runner.
        
        Args:
            data_dir: Directory containing the CSV datasets
        """
        self.data_dir = Path(data_dir)
        self.results = []
        
        # Define hyperparameter grid for tuning
        self.param_grid = {
            'criterion': ['gini', 'entropy'],
            'splitter': ['best', 'random'],
            'max_depth': [None, 10, 20, 30, 50, 100],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4],
            'max_features': [None, 'sqrt', 'log2']
        }
    
    def load_dataset(self, clauses, examples):
        """
        Load train, validation, and test sets for a specific configuration.
        
        Args:
            clauses: Number of clauses (300, 500, 1000, 1500, 1800)
            examples: Number of examples (100, 1000, 5000)
            
        Returns:
            Dictionary with train, valid, and test DataFrames
        """
        base_name = f"c{clauses}_d{examples}"
        
        train_file = self.data_dir / f"train_{base_name}.csv"
        valid_file = self.data_dir / f"valid_{base_name}.csv"
        test_file = self.data_dir / f"test_{base_name}.csv"
        
        # Check if files exist
        for file in [train_file, valid_file, test_file]:
            if not file.exists():
                raise FileNotFoundError(f"Dataset file not found: {file}")
        
        # Load datasets
        train_df = pd.read_csv(train_file, header=None)
        valid_df = pd.read_csv(valid_file, header=None)
        test_df = pd.read_csv(test_file, header=None)
        
        return {
            'train': train_df,
            'valid': valid_df,
            'test': test_df
        }
    
    def prepare_data(self, df):
        """
        Split features and labels from a DataFrame.
        
        Args:
            df: DataFrame with features and label in last column
            
        Returns:
            X (features), y (labels)
        """
        X = df.iloc[:, :-1].values
        y = df.iloc[:, -1].values
        return X, y
    
    def tune_hyperparameters(self, X_train, y_train, X_valid, y_valid):
        """
        Tune hyperparameters using validation set.
        
        Args:
            X_train: Training features
            y_train: Training labels
            X_valid: Validation features
            y_valid: Validation labels
            
        Returns:
            Best parameters and best validation score
        """
        print("  Tuning hyperparameters...")
        
        # Use GridSearchCV with custom scoring
        dt = DecisionTreeClassifier(random_state=42)
        
        # Perform grid search with cross-validation on training set
        grid_search = GridSearchCV(
            dt,
            self.param_grid,
            cv=5,
            scoring='f1',
            n_jobs=-1,
            verbose=0
        )
        
        grid_search.fit(X_train, y_train)
        
        # Evaluate best model on validation set
        best_model = grid_search.best_estimator_
        valid_score = f1_score(y_valid, best_model.predict(X_valid))
        
        print(f"  Best cross-validation F1: {grid_search.best_score_:.4f}")
        print(f"  Validation F1: {valid_score:.4f}")
        
        return grid_search.best_params_, valid_score
    
    def train_final_model(self, X_train, y_train, X_valid, y_valid, best_params):
        """
        Train final model on combined train+validation set.
        
        Args:
            X_train: Training features
            y_train: Training labels
            X_valid: Validation features
            y_valid: Validation labels
            best_params: Best hyperparameters from tuning
            
        Returns:
            Trained model
        """
        print("  Training final model on combined train+valid set...")
        
        # Combine training and validation sets
        X_combined = np.vstack([X_train, X_valid])
        y_combined = np.hstack([y_train, y_valid])
        
        # Train final model
        final_model = DecisionTreeClassifier(**best_params, random_state=42)
        final_model.fit(X_combined, y_combined)
        
        return final_model
    
    def evaluate_model(self, model, X_test, y_test):
        """
        Evaluate model on test set.
        
        Args:
            model: Trained model
            X_test: Test features
            y_test: Test labels
            
        Returns:
            Dictionary with accuracy and F1 score
        """
        y_pred = model.predict(X_test)
        
        accuracy = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        
        return {
            'accuracy': accuracy,
            'f1_score': f1
        }
    
    def run_experiment(self, clauses, examples):
        """
        Run complete experiment for one dataset configuration.
        
        Args:
            clauses: Number of clauses
            examples: Number of examples
            
        Returns:
            Dictionary with results
        """
        print(f"\n{'='*60}")
        print(f"Experiment: {clauses} clauses, {examples} examples")
        print(f"{'='*60}")
        
        # Load data
        print("Loading datasets...")
        data = self.load_dataset(clauses, examples)
        
        # Prepare data
        X_train, y_train = self.prepare_data(data['train'])
        X_valid, y_valid = self.prepare_data(data['valid'])
        X_test, y_test = self.prepare_data(data['test'])
        
        print(f"  Train size: {X_train.shape}")
        print(f"  Valid size: {X_valid.shape}")
        print(f"  Test size: {X_test.shape}")
        
        # Tune hyperparameters
        best_params, valid_score = self.tune_hyperparameters(
            X_train, y_train, X_valid, y_valid
        )
        
        # Train final model
        final_model = self.train_final_model(
            X_train, y_train, X_valid, y_valid, best_params
        )
        
        # Evaluate on test set
        print("  Evaluating on test set...")
        test_metrics = self.evaluate_model(final_model, X_test, y_test)
        
        # Compile results
        result = {
            'clauses': clauses,
            'examples': examples,
            'n_features': X_train.shape[1],
            'best_params': best_params,
            'validation_f1': valid_score,
            'test_accuracy': test_metrics['accuracy'],
            'test_f1_score': test_metrics['f1_score']
        }
        
        print(f"\n  Results:")
        print(f"    Test Accuracy: {test_metrics['accuracy']:.4f}")
        print(f"    Test F1 Score: {test_metrics['f1_score']:.4f}")
        
        return result
    
    def discover_datasets(self):
        """
        Discover available datasets in the data directory.
        
        Returns:
            List of tuples (clauses, examples) for available datasets
        """
        available_datasets = []
        
        # Check all possible combinations
        clause_configs = [300, 500, 1000, 1500, 1800]
        example_configs = [100, 1000, 5000]
        
        print("\nScanning for available datasets...")
        print("-" * 60)
        
        for clauses in clause_configs:
            for examples in example_configs:
                base_name = f"c{clauses}_d{examples}"
                train_file = self.data_dir / f"train_{base_name}.csv"
                valid_file = self.data_dir / f"valid_{base_name}.csv"
                test_file = self.data_dir / f"test_{base_name}.csv"
                
                # Check if all three files exist
                if train_file.exists() and valid_file.exists() and test_file.exists():
                    available_datasets.append((clauses, examples))
                    print(f"✓ Found: {clauses} clauses, {examples} examples")
                else:
                    missing = []
                    if not train_file.exists():
                        missing.append("train")
                    if not valid_file.exists():
                        missing.append("valid")
                    if not test_file.exists():
                        missing.append("test")
                    print(f"✗ Missing ({clauses} clauses, {examples} examples): {', '.join(missing)}")
        
        print("-" * 60)
        print(f"Total available datasets: {len(available_datasets)}/15\n")
        
        return available_datasets
    
    def run_all_experiments(self):
        """
        Run experiments on all dataset configurations.
        """
        print("Starting Decision Tree Experiments on CNF Datasets")
        print("=" * 60)
        print(f"Data directory: {self.data_dir.absolute()}")
        
        # Discover available datasets
        available_datasets = self.discover_datasets()
        
        if not available_datasets:
            print("\nERROR: No complete datasets found!")
            print(f"Please check that CSV files are in: {self.data_dir.absolute()}")
            print("Expected filename format: train_c[clauses]_d[examples].csv")
            return
        
        # Run experiments on available datasets
        for clauses, examples in available_datasets:
            try:
                result = self.run_experiment(clauses, examples)
                self.results.append(result)
            except Exception as e:
                print(f"\nError in experiment ({clauses} clauses, {examples} examples): {e}")
                import traceback
                traceback.print_exc()
        
        if self.results:
            self.save_results()
            self.print_summary()
        else:
            print("\nNo experiments completed successfully.")
    
    def save_results(self, filename='dt_results.json'):
        """Save results to JSON file."""
        with open(filename, 'w') as f:
            json.dump(self.results, f, indent=2)
        print(f"\nResults saved to {filename}")
    
    def print_summary(self):
        """Print summary table of all results."""
        print("\n" + "="*80)
        print("SUMMARY OF ALL EXPERIMENTS")
        print("="*80)
        print(f"{'Clauses':<10} {'Examples':<10} {'Features':<10} {'Test Acc':<12} {'Test F1':<12}")
        print("-"*80)
        
        for result in self.results:
            print(f"{result['clauses']:<10} {result['examples']:<10} "
                  f"{result['n_features']:<10} "
                  f"{result['test_accuracy']:<12.4f} "
                  f"{result['test_f1_score']:<12.4f}")
        
        print("="*80)
        
        # Print best parameters for each configuration
        print("\nBEST HYPERPARAMETERS FOR EACH CONFIGURATION:")
        print("="*80)
        for result in self.results:
            print(f"\nClauses: {result['clauses']}, Examples: {result['examples']}")
            for param, value in result['best_params'].items():
                print(f"  {param}: {value}")


def main():
    """Main execution function."""
    
    # Initialize experiment runner
    # Update 'data_dir' to point to your all_data folder
    experiment = CNFDatasetExperiment(data_dir='./all_data')
    
    # Run all experiments
    experiment.run_all_experiments()
    
    # Results are automatically saved and printed


if __name__ == "__main__":
    main()

Starting Decision Tree Experiments on CNF Datasets
Data directory: c:\utd\sem1\machinelearning\project2\project2_data\all_data

Scanning for available datasets...
------------------------------------------------------------
✓ Found: 300 clauses, 100 examples
✓ Found: 300 clauses, 1000 examples
✓ Found: 300 clauses, 5000 examples
✓ Found: 500 clauses, 100 examples
✓ Found: 500 clauses, 1000 examples
✓ Found: 500 clauses, 5000 examples
✓ Found: 1000 clauses, 100 examples
✓ Found: 1000 clauses, 1000 examples
✓ Found: 1000 clauses, 5000 examples
✓ Found: 1500 clauses, 100 examples
✓ Found: 1500 clauses, 1000 examples
✓ Found: 1500 clauses, 5000 examples
✓ Found: 1800 clauses, 100 examples
✓ Found: 1800 clauses, 1000 examples
✓ Found: 1800 clauses, 5000 examples
------------------------------------------------------------
Total available datasets: 15/15


Experiment: 300 clauses, 100 examples
Loading datasets...
  Train size: (200, 500)
  Valid size: (200, 500)
  Test size: (200, 500)
  Tun

In [7]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import RandomizedSearchCV
import json
from pathlib import Path
import time

class BaggingCNFExperiment:
    """
    Experiment runner for Bagging with Decision Tree classification on CNF datasets.
    Optimized to run under 10 minutes.
    """
    
    def __init__(self, data_dir='./all_data'):
        self.data_dir = Path(data_dir)
        self.results = []
        
        # Highly optimized hyperparameter distributions
        self.param_distributions = {
            # Bagging-specific parameters
            'n_estimators': [25, 50, 100],
            'max_samples': [0.8, 1.0],
            'max_features': [0.8, 1.0],
            'bootstrap': [True],
            'bootstrap_features': [False],
            
            # Base estimator (Decision Tree) parameters  
            'estimator__criterion': ['gini', 'entropy'],
            'estimator__max_depth': [None, 20, 40],
            'estimator__min_samples_split': [2, 10],
            'estimator__min_samples_leaf': [1, 4],
        }
        
        # Reduced iterations for speed
        self.n_iter = 20  # Reduced from 50
    
    def discover_datasets(self):
        available_datasets = []
        clause_configs = [300, 500, 1000, 1500, 1800]
        example_configs = [100, 1000, 5000]
        
        print("\nScanning for available datasets...")
        print("-" * 60)
        
        for clauses in clause_configs:
            for examples in example_configs:
                base_name = f"c{clauses}_d{examples}"
                train_file = self.data_dir / f"train_{base_name}.csv"
                valid_file = self.data_dir / f"valid_{base_name}.csv"
                test_file = self.data_dir / f"test_{base_name}.csv"
                
                if train_file.exists() and valid_file.exists() and test_file.exists():
                    available_datasets.append((clauses, examples))
                    print(f"Found: {clauses} clauses, {examples} examples")
        
        print("-" * 60)
        print(f"Total available datasets: {len(available_datasets)}/15\n")
        
        return available_datasets
    
    def load_dataset(self, clauses, examples):
        base_name = f"c{clauses}_d{examples}"
        
        train_file = self.data_dir / f"train_{base_name}.csv"
        valid_file = self.data_dir / f"valid_{base_name}.csv"
        test_file = self.data_dir / f"test_{base_name}.csv"
        
        train_df = pd.read_csv(train_file, header=None)
        valid_df = pd.read_csv(valid_file, header=None)
        test_df = pd.read_csv(test_file, header=None)
        
        return {
            'train': train_df,
            'valid': valid_df,
            'test': test_df
        }
    
    def prepare_data(self, df):
        X = df.iloc[:, :-1].values
        y = df.iloc[:, -1].values
        return X, y
    
    def tune_hyperparameters(self, X_train, y_train, X_valid, y_valid):
        print("  Tuning hyperparameters...")
        start_time = time.time()
        
        base_estimator = DecisionTreeClassifier(random_state=42)
        bagging = BaggingClassifier(
            estimator=base_estimator,
            random_state=42,
            n_jobs=-1
        )
        
        # Using 2-fold CV for maximum speed
        random_search = RandomizedSearchCV(
            bagging,
            self.param_distributions,
            n_iter=self.n_iter,
            cv=2,  # Reduced from 3 to 2
            scoring='f1',
            n_jobs=-1,
            verbose=0,
            random_state=42
        )
        
        random_search.fit(X_train, y_train)
        
        best_model = random_search.best_estimator_
        valid_score = f1_score(y_valid, best_model.predict(X_valid))
        
        elapsed_time = time.time() - start_time
        print(f"  Best CV F1: {random_search.best_score_:.4f}, Validation F1: {valid_score:.4f}")
        print(f"  Tuning time: {elapsed_time:.1f}s")
        
        return random_search.best_params_, valid_score
    
    def train_final_model(self, X_train, y_train, X_valid, y_valid, best_params):
        print("  Training final model...")
        
        X_combined = np.vstack([X_train, X_valid])
        y_combined = np.hstack([y_train, y_valid])
        
        bagging_params = {}
        estimator_params = {}
        
        for key, value in best_params.items():
            if key.startswith('estimator__'):
                estimator_params[key.replace('estimator__', '')] = value
            else:
                bagging_params[key] = value
        
        base_estimator = DecisionTreeClassifier(**estimator_params, random_state=42)
        final_model = BaggingClassifier(
            estimator=base_estimator,
            **bagging_params,
            random_state=42,
            n_jobs=-1
        )
        final_model.fit(X_combined, y_combined)
        
        return final_model
    
    def evaluate_model(self, model, X_test, y_test):
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        
        return {
            'accuracy': accuracy,
            'f1_score': f1
        }
    
    def run_experiment(self, clauses, examples):
        print(f"\n{'='*60}")
        print(f"Experiment: {clauses} clauses, {examples} examples")
        print(f"{'='*60}")
        
        data = self.load_dataset(clauses, examples)
        
        X_train, y_train = self.prepare_data(data['train'])
        X_valid, y_valid = self.prepare_data(data['valid'])
        X_test, y_test = self.prepare_data(data['test'])
        
        print(f"  Train: {X_train.shape}, Valid: {X_valid.shape}, Test: {X_test.shape}")
        
        best_params, valid_score = self.tune_hyperparameters(
            X_train, y_train, X_valid, y_valid
        )
        
        final_model = self.train_final_model(
            X_train, y_train, X_valid, y_valid, best_params
        )
        
        print("  Evaluating on test set...")
        test_metrics = self.evaluate_model(final_model, X_test, y_test)
        
        result = {
            'clauses': clauses,
            'examples': examples,
            'n_features': X_train.shape[1],
            'best_params': best_params,
            'validation_f1': valid_score,
            'test_accuracy': test_metrics['accuracy'],
            'test_f1_score': test_metrics['f1_score']
        }
        
        print(f"  Test Accuracy: {test_metrics['accuracy']:.4f}")
        print(f"  Test F1 Score: {test_metrics['f1_score']:.4f}")
        
        return result
    
    def run_all_experiments(self):
        print("\nBAGGING WITH DECISION TREES - CNF CLASSIFICATION")
        print("="*60)
        print(f"Data directory: {self.data_dir.absolute()}")
        print("Time limit: 10 minutes")
        
        total_start_time = time.time()
        time_limit = 10 * 60
        
        available_datasets = self.discover_datasets()
        
        if not available_datasets:
            print("\nERROR: No complete datasets found!")
            return
        
        print(f"Processing {len(available_datasets)} datasets\n")
        
        for idx, (clauses, examples) in enumerate(available_datasets, 1):
            elapsed = time.time() - total_start_time
            remaining = time_limit - elapsed
            
            print(f"\n[Dataset {idx}/{len(available_datasets)}] Elapsed: {elapsed/60:.1f}m, Remaining: {remaining/60:.1f}m")
            
            if remaining < 30:
                print("Warning: Less than 30 seconds remaining, stopping.")
                break
            
            try:
                result = self.run_experiment(clauses, examples)
                self.results.append(result)
            except Exception as e:
                print(f"Error: {e}")
        
        total_time = time.time() - total_start_time
        print(f"\n{'='*60}")
        print(f"Total execution time: {total_time/60:.2f} minutes")
        print(f"{'='*60}")
        
        if self.results:
            self.save_results()
            self.print_summary()
        else:
            print("No experiments completed.")
    
    def save_results(self, filename='bagging_results.json'):
        with open(filename, 'w') as f:
            json.dump(self.results, f, indent=2)
        print(f"\nResults saved to {filename}")
    
    def print_summary(self):
        print("\n" + "="*80)
        print("SUMMARY - BAGGING WITH DECISION TREES")
        print("="*80)
        print(f"{'Clauses':<10} {'Examples':<10} {'Features':<10} {'Test Acc':<12} {'Test F1':<12}")
        print("-"*80)
        
        for result in self.results:
            print(f"{result['clauses']:<10} {result['examples']:<10} "
                  f"{result['n_features']:<10} "
                  f"{result['test_accuracy']:<12.4f} "
                  f"{result['test_f1_score']:<12.4f}")
        
        print("="*80)
        
        print("\n" + "="*80)
        print("BEST HYPERPARAMETERS FOR EACH CONFIGURATION")
        print("="*80)
        
        for result in self.results:
            print(f"\nConfiguration: {result['clauses']} clauses, {result['examples']} examples")
            print("-"*80)
            
            bagging_params = {}
            estimator_params = {}
            
            for param, value in result['best_params'].items():
                if param.startswith('estimator__'):
                    estimator_params[param] = value
                else:
                    bagging_params[param] = value
            
            print("Bagging Parameters:")
            for param, value in sorted(bagging_params.items()):
                print(f"  {param}: {value}")
            
            print("Base Estimator (Decision Tree) Parameters:")
            for param, value in sorted(estimator_params.items()):
                param_clean = param.replace('estimator__', '')
                print(f"  {param_clean}: {value}")
            
            print(f"Performance:")
            print(f"  Validation F1: {result['validation_f1']:.4f}")
            print(f"  Test Accuracy: {result['test_accuracy']:.4f}")
            print(f"  Test F1 Score: {result['test_f1_score']:.4f}")


def main():
    print("\nEXPERIMENT 2: BAGGING WITH DECISION TREES")
    print("CNF Boolean Formula Classification")
    print("="*60)
    
    experiment = BaggingCNFExperiment(data_dir='./all_data')
    experiment.run_all_experiments()
    
    print("\nEXPERIMENT COMPLETED")
    print("="*60)


if __name__ == "__main__":
    main()


EXPERIMENT 2: BAGGING WITH DECISION TREES
CNF Boolean Formula Classification

BAGGING WITH DECISION TREES - CNF CLASSIFICATION
Data directory: c:\utd\sem1\machinelearning\project2\project2_data\all_data
Time limit: 10 minutes

Scanning for available datasets...
------------------------------------------------------------
Found: 300 clauses, 100 examples
Found: 300 clauses, 1000 examples
Found: 300 clauses, 5000 examples
Found: 500 clauses, 100 examples
Found: 500 clauses, 1000 examples
Found: 500 clauses, 5000 examples
Found: 1000 clauses, 100 examples
Found: 1000 clauses, 1000 examples
Found: 1000 clauses, 5000 examples
Found: 1500 clauses, 100 examples
Found: 1500 clauses, 1000 examples
Found: 1500 clauses, 5000 examples
Found: 1800 clauses, 100 examples
Found: 1800 clauses, 1000 examples
Found: 1800 clauses, 5000 examples
------------------------------------------------------------
Total available datasets: 15/15

Processing 15 datasets


[Dataset 1/15] Elapsed: 0.0m, Remaining: 10

In [5]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import RandomizedSearchCV
import json
from pathlib import Path
import time

class RandomForestCNFExperiment:
    """
    Experiment runner for Random Forest classification on CNF datasets.
    Optimized for best accuracy and F1 scores with reasonable runtime.
    """
    
    def __init__(self, data_dir='./all_data'):
        self.data_dir = Path(data_dir)
        self.results = []
        
        # Comprehensive hyperparameter distributions for best performance
        self.param_distributions = {
            'n_estimators': [100, 200, 300],
            'criterion': ['gini', 'entropy'],
            'max_depth': [None, 30, 50, 70],
            'min_samples_split': [2, 5, 10, 15],
            'min_samples_leaf': [1, 2, 4, 8],
            'max_features': ['sqrt', 'log2'],
            'bootstrap': [True],
            'max_samples': [0.8, 0.9, 1.0],
            'min_impurity_decrease': [0.0, 0.0001, 0.001],
            'class_weight': [None, 'balanced', 'balanced_subsample'],
        }
    
    def discover_datasets(self):
        available_datasets = []
        clause_configs = [300, 500, 1000, 1500, 1800]
        example_configs = [100, 1000, 5000]
        
        print("\nScanning for available datasets...")
        print("-" * 60)
        
        for clauses in clause_configs:
            for examples in example_configs:
                base_name = f"c{clauses}_d{examples}"
                train_file = self.data_dir / f"train_{base_name}.csv"
                valid_file = self.data_dir / f"valid_{base_name}.csv"
                test_file = self.data_dir / f"test_{base_name}.csv"
                
                if train_file.exists() and valid_file.exists() and test_file.exists():
                    available_datasets.append((clauses, examples))
                    print(f"Found: {clauses} clauses, {examples} examples")
        
        print("-" * 60)
        print(f"Total available datasets: {len(available_datasets)}/15\n")
        
        return available_datasets
    
    def load_dataset(self, clauses, examples):
        base_name = f"c{clauses}_d{examples}"
        
        train_file = self.data_dir / f"train_{base_name}.csv"
        valid_file = self.data_dir / f"valid_{base_name}.csv"
        test_file = self.data_dir / f"test_{base_name}.csv"
        
        train_df = pd.read_csv(train_file, header=None)
        valid_df = pd.read_csv(valid_file, header=None)
        test_df = pd.read_csv(test_file, header=None)
        
        return {
            'train': train_df,
            'valid': valid_df,
            'test': test_df
        }
    
    def prepare_data(self, df):
        X = df.iloc[:, :-1].values
        y = df.iloc[:, -1].values
        return X, y
    
    def tune_hyperparameters(self, X_train, y_train, X_valid, y_valid, examples):
        print("  Tuning hyperparameters with extensive search...")
        start_time = time.time()
        
        # More iterations for better hyperparameter search
        # Adaptive based on dataset size for efficiency
        if examples == 100:
            n_iter = 40
            cv = 5
        elif examples == 1000:
            n_iter = 35
            cv = 5
        else:  # 5000
            n_iter = 30
            cv = 3  # Use 3-fold for large datasets
        
        rf = RandomForestClassifier(random_state=42, n_jobs=-1, warm_start=False)
        
        random_search = RandomizedSearchCV(
            rf,
            self.param_distributions,
            n_iter=n_iter,
            cv=cv,
            scoring='f1',
            n_jobs=-1,
            verbose=0,
            random_state=42,
            return_train_score=True
        )
        
        random_search.fit(X_train, y_train)
        
        best_model = random_search.best_estimator_
        
        # Evaluate on validation set
        valid_pred = best_model.predict(X_valid)
        valid_score = f1_score(y_valid, valid_pred)
        valid_acc = accuracy_score(y_valid, valid_pred)
        
        # Also check training score to detect overfitting
        train_pred = best_model.predict(X_train)
        train_score = f1_score(y_train, train_pred)
        
        elapsed_time = time.time() - start_time
        print(f"  Best CV F1: {random_search.best_score_:.4f}")
        print(f"  Train F1: {train_score:.4f}, Validation F1: {valid_score:.4f}")
        print(f"  Validation Accuracy: {valid_acc:.4f}")
        print(f"  Tuning time: {elapsed_time:.1f}s")
        
        # Check for overfitting
        if train_score - valid_score > 0.1:
            print(f"  Note: Possible overfitting detected (train-valid gap: {train_score - valid_score:.4f})")
        
        return random_search.best_params_, valid_score
    
    def train_final_model(self, X_train, y_train, X_valid, y_valid, best_params):
        print("  Training final model on combined train+valid...")
        
        X_combined = np.vstack([X_train, X_valid])
        y_combined = np.hstack([y_train, y_valid])
        
        final_model = RandomForestClassifier(**best_params, random_state=42, n_jobs=-1)
        final_model.fit(X_combined, y_combined)
        
        return final_model
    
    def evaluate_model(self, model, X_test, y_test):
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        
        # Additional metrics
        from sklearn.metrics import precision_score, recall_score
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        
        return {
            'accuracy': accuracy,
            'f1_score': f1,
            'precision': precision,
            'recall': recall
        }
    
    def run_experiment(self, clauses, examples):
        print(f"\n{'='*60}")
        print(f"Experiment: {clauses} clauses, {examples} examples")
        print(f"{'='*60}")
        
        data = self.load_dataset(clauses, examples)
        
        X_train, y_train = self.prepare_data(data['train'])
        X_valid, y_valid = self.prepare_data(data['valid'])
        X_test, y_test = self.prepare_data(data['test'])
        
        print(f"  Train: {X_train.shape}, Valid: {X_valid.shape}, Test: {X_test.shape}")
        
        # Show class distribution
        train_pos = np.sum(y_train) / len(y_train)
        test_pos = np.sum(y_test) / len(y_test)
        print(f"  Class balance - Train: {train_pos:.2%} positive, Test: {test_pos:.2%} positive")
        
        best_params, valid_score = self.tune_hyperparameters(
            X_train, y_train, X_valid, y_valid, examples
        )
        
        final_model = self.train_final_model(
            X_train, y_train, X_valid, y_valid, best_params
        )
        
        print("  Evaluating on test set...")
        test_metrics = self.evaluate_model(final_model, X_test, y_test)
        
        result = {
            'clauses': clauses,
            'examples': examples,
            'n_features': X_train.shape[1],
            'best_params': best_params,
            'validation_f1': valid_score,
            'test_accuracy': test_metrics['accuracy'],
            'test_f1_score': test_metrics['f1_score'],
            'test_precision': test_metrics['precision'],
            'test_recall': test_metrics['recall']
        }
        
        print(f"  Test Results:")
        print(f"    Accuracy:  {test_metrics['accuracy']:.4f}")
        print(f"    F1 Score:  {test_metrics['f1_score']:.4f}")
        print(f"    Precision: {test_metrics['precision']:.4f}")
        print(f"    Recall:    {test_metrics['recall']:.4f}")
        
        return result
    
    def run_all_experiments(self):
        print("\nRANDOM FOREST CLASSIFIER - CNF CLASSIFICATION")
        print("="*60)
        print(f"Data directory: {self.data_dir.absolute()}")
        print("Focus: Best accuracy and F1 scores")
        
        total_start_time = time.time()
        
        available_datasets = self.discover_datasets()
        
        if not available_datasets:
            print("\nERROR: No complete datasets found!")
            return
        
        print(f"Processing {len(available_datasets)} datasets\n")
        
        for idx, (clauses, examples) in enumerate(available_datasets, 1):
            elapsed = time.time() - total_start_time
            
            print(f"\n[Dataset {idx}/{len(available_datasets)}] Elapsed: {elapsed/60:.1f}m")
            
            try:
                dataset_start = time.time()
                result = self.run_experiment(clauses, examples)
                self.results.append(result)
                dataset_time = time.time() - dataset_start
                print(f"  Dataset completed in {dataset_time:.1f}s")
            except Exception as e:
                print(f"Error: {e}")
                import traceback
                traceback.print_exc()
        
        total_time = time.time() - total_start_time
        print(f"\n{'='*60}")
        print(f"Total execution time: {total_time/60:.2f} minutes")
        print(f"Completed {len(self.results)}/{len(available_datasets)} datasets")
        print(f"{'='*60}")
        
        if self.results:
            self.save_results()
            self.print_summary()
        else:
            print("No experiments completed.")
    
    def save_results(self, filename='random_forest_results.json'):
        with open(filename, 'w') as f:
            json.dump(self.results, f, indent=2)
        print(f"\nResults saved to {filename}")
    
    def print_summary(self):
        print("\n" + "="*90)
        print("SUMMARY - RANDOM FOREST CLASSIFIER")
        print("="*90)
        print(f"{'Clauses':<10} {'Examples':<10} {'Features':<10} {'Test Acc':<12} {'Test F1':<12} {'Precision':<12} {'Recall':<12}")
        print("-"*90)
        
        for result in self.results:
            print(f"{result['clauses']:<10} {result['examples']:<10} "
                  f"{result['n_features']:<10} "
                  f"{result['test_accuracy']:<12.4f} "
                  f"{result['test_f1_score']:<12.4f} "
                  f"{result['test_precision']:<12.4f} "
                  f"{result['test_recall']:<12.4f}")
        
        print("="*90)
        
        print("\n" + "="*80)
        print("BEST HYPERPARAMETERS FOR EACH CONFIGURATION")
        print("="*80)
        
        for result in self.results:
            print(f"\nConfiguration: {result['clauses']} clauses, {result['examples']} examples")
            print("-"*80)
            
            print("Random Forest Parameters:")
            for param, value in sorted(result['best_params'].items()):
                print(f"  {param}: {value}")
            
            print(f"\nPerformance Metrics:")
            print(f"  Validation F1: {result['validation_f1']:.4f}")
            print(f"  Test Accuracy:  {result['test_accuracy']:.4f}")
            print(f"  Test F1 Score:  {result['test_f1_score']:.4f}")
            print(f"  Test Precision: {result['test_precision']:.4f}")
            print(f"  Test Recall:    {result['test_recall']:.4f}")


def main():
    print("\nEXPERIMENT 3: RANDOM FOREST CLASSIFIER")
    print("CNF Boolean Formula Classification")
    print("="*60)
    
    experiment = RandomForestCNFExperiment(data_dir='./all_data')
    experiment.run_all_experiments()
    
    print("\nEXPERIMENT COMPLETED")
    print("="*60)


if __name__ == "__main__":
    main()


EXPERIMENT 3: RANDOM FOREST CLASSIFIER
CNF Boolean Formula Classification

RANDOM FOREST CLASSIFIER - CNF CLASSIFICATION
Data directory: c:\utd\sem1\machinelearning\project2\project2_data\all_data
Focus: Best accuracy and F1 scores

Scanning for available datasets...
------------------------------------------------------------
Found: 300 clauses, 100 examples
Found: 300 clauses, 1000 examples
Found: 300 clauses, 5000 examples
Found: 500 clauses, 100 examples
Found: 500 clauses, 1000 examples
Found: 500 clauses, 5000 examples
Found: 1000 clauses, 100 examples
Found: 1000 clauses, 1000 examples
Found: 1000 clauses, 5000 examples
Found: 1500 clauses, 100 examples
Found: 1500 clauses, 1000 examples
Found: 1500 clauses, 5000 examples
Found: 1800 clauses, 100 examples
Found: 1800 clauses, 1000 examples
Found: 1800 clauses, 5000 examples
------------------------------------------------------------
Total available datasets: 15/15

Processing 15 datasets


[Dataset 1/15] Elapsed: 0.0m

Experim

In [20]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import RandomizedSearchCV
import json
from pathlib import Path
import time

class GradientBoostingCNFExperiment:
    """
    Experiment runner for Gradient Boosting classification on CNF datasets.
    Optimized for speed while maintaining good performance.
    """
    
    def __init__(self, data_dir='./all_data'):
        self.data_dir = Path(data_dir)
        self.results = []
        
        # Reduced hyperparameter space for faster search
        self.param_distributions = {
            'n_estimators': [50, 100, 150],
            'learning_rate': [0.05, 0.1, 0.2],
            'max_depth': [3, 5, 7],
            'min_samples_split': [2, 10],
            'min_samples_leaf': [1, 4],
            'max_features': ['sqrt', 'log2'],
            'subsample': [0.8, 1.0],
        }
    
    def discover_datasets(self):
        available_datasets = []
        clause_configs = [300, 500, 1000, 1500, 1800]
        example_configs = [100, 1000, 5000]
        
        print("\nScanning for available datasets...")
        print("-" * 60)
        
        for clauses in clause_configs:
            for examples in example_configs:
                base_name = f"c{clauses}_d{examples}"
                train_file = self.data_dir / f"train_{base_name}.csv"
                valid_file = self.data_dir / f"valid_{base_name}.csv"
                test_file = self.data_dir / f"test_{base_name}.csv"
                
                if train_file.exists() and valid_file.exists() and test_file.exists():
                    available_datasets.append((clauses, examples))
                    print(f"✓ Found: {clauses} clauses, {examples} examples")
        
        print("-" * 60)
        print(f"Total available datasets: {len(available_datasets)}/15\n")
        
        return available_datasets
    
    def load_dataset(self, clauses, examples):
        base_name = f"c{clauses}_d{examples}"
        
        train_file = self.data_dir / f"train_{base_name}.csv"
        valid_file = self.data_dir / f"valid_{base_name}.csv"
        test_file = self.data_dir / f"test_{base_name}.csv"
        
        train_df = pd.read_csv(train_file, header=None)
        valid_df = pd.read_csv(valid_file, header=None)
        test_df = pd.read_csv(test_file, header=None)
        
        return {
            'train': train_df,
            'valid': valid_df,
            'test': test_df
        }
    
    def prepare_data(self, df):
        X = df.iloc[:, :-1].values
        y = df.iloc[:, -1].values
        return X, y
    
    def tune_hyperparameters(self, X_train, y_train, X_valid, y_valid, examples):
        print("  Tuning hyperparameters...")
        start_time = time.time()
        
        # Significantly reduced iterations for speed
        if examples == 100:
            n_iter = 15
            cv = 3
        elif examples == 1000:
            n_iter = 12
            cv = 3
        else:  # 5000
            n_iter = 10
            cv = 2
        
        gb = GradientBoostingClassifier(random_state=42, verbose=0)
        
        random_search = RandomizedSearchCV(
            gb,
            self.param_distributions,
            n_iter=n_iter,
            cv=cv,
            scoring='f1',
            n_jobs=-1,
            verbose=0,
            random_state=42,
            return_train_score=False  # Don't compute train scores to save time
        )
        
        random_search.fit(X_train, y_train)
        
        best_model = random_search.best_estimator_
        
        # Evaluate on validation set
        valid_pred = best_model.predict(X_valid)
        valid_score = f1_score(y_valid, valid_pred)
        valid_acc = accuracy_score(y_valid, valid_pred)
        
        elapsed_time = time.time() - start_time
        print(f"  Best CV F1: {random_search.best_score_:.4f}")
        print(f"  Validation F1: {valid_score:.4f}, Validation Acc: {valid_acc:.4f}")
        print(f"  Tuning time: {elapsed_time:.1f}s")
        
        return random_search.best_params_, valid_score
    
    def train_final_model(self, X_train, y_train, X_valid, y_valid, best_params):
        print("  Training final model on combined train+valid...")
        
        X_combined = np.vstack([X_train, X_valid])
        y_combined = np.hstack([y_train, y_valid])
        
        final_model = GradientBoostingClassifier(**best_params, random_state=42, verbose=0)
        final_model.fit(X_combined, y_combined)
        
        return final_model
    
    def evaluate_model(self, model, X_test, y_test):
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        
        # Additional metrics
        from sklearn.metrics import precision_score, recall_score
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        
        return {
            'accuracy': accuracy,
            'f1_score': f1,
            'precision': precision,
            'recall': recall
        }
    
    def run_experiment(self, clauses, examples):
        print(f"\n{'='*60}")
        print(f"Experiment: {clauses} clauses, {examples} examples")
        print(f"{'='*60}")
        
        data = self.load_dataset(clauses, examples)
        
        X_train, y_train = self.prepare_data(data['train'])
        X_valid, y_valid = self.prepare_data(data['valid'])
        X_test, y_test = self.prepare_data(data['test'])
        
        print(f"  Dataset shapes: Train={X_train.shape}, Valid={X_valid.shape}, Test={X_test.shape}")
        
        best_params, valid_score = self.tune_hyperparameters(
            X_train, y_train, X_valid, y_valid, examples
        )
        
        final_model = self.train_final_model(
            X_train, y_train, X_valid, y_valid, best_params
        )
        
        print("  Evaluating on test set...")
        test_metrics = self.evaluate_model(final_model, X_test, y_test)
        
        result = {
            'clauses': clauses,
            'examples': examples,
            'n_features': X_train.shape[1],
            'best_params': best_params,
            'validation_f1': valid_score,
            'test_accuracy': test_metrics['accuracy'],
            'test_f1_score': test_metrics['f1_score'],
            'test_precision': test_metrics['precision'],
            'test_recall': test_metrics['recall']
        }
        
        print(f"  Test Results:")
        print(f"    Accuracy:  {test_metrics['accuracy']:.4f}")
        print(f"    F1 Score:  {test_metrics['f1_score']:.4f}")
        print(f"    Precision: {test_metrics['precision']:.4f}")
        print(f"    Recall:    {test_metrics['recall']:.4f}")
        
        return result
    
    def run_all_experiments(self):
        print("\nGRADIENT BOOSTING CLASSIFIER - CNF CLASSIFICATION")
        print("="*60)
        print(f"Data directory: {self.data_dir.absolute()}")
        print("Optimized for speed (<10 minutes target)")
        
        total_start_time = time.time()
        
        available_datasets = self.discover_datasets()
        
        if not available_datasets:
            print("\nERROR: No complete datasets found!")
            return
        
        print(f"Processing {len(available_datasets)} datasets\n")
        
        for idx, (clauses, examples) in enumerate(available_datasets, 1):
            elapsed = time.time() - total_start_time
            
            print(f"\n[{idx}/{len(available_datasets)}] Elapsed: {elapsed/60:.1f}m | "
                  f"ETA: {(elapsed/idx)*(len(available_datasets)-idx)/60:.1f}m")
            
            try:
                dataset_start = time.time()
                result = self.run_experiment(clauses, examples)
                self.results.append(result)
                dataset_time = time.time() - dataset_start
                print(f"  ✓ Completed in {dataset_time:.1f}s")
            except Exception as e:
                print(f"  ✗ Error: {e}")
                import traceback
                traceback.print_exc()
        
        total_time = time.time() - total_start_time
        print(f"\n{'='*60}")
        print(f"Total execution time: {total_time/60:.2f} minutes")
        print(f"Completed {len(self.results)}/{len(available_datasets)} datasets")
        print(f"{'='*60}")
        
        if self.results:
            self.save_results()
            self.print_summary()
        else:
            print("No experiments completed.")
    
    def save_results(self, filename='gradient_boosting_results.json'):
        with open(filename, 'w') as f:
            json.dump(self.results, f, indent=2)
        print(f"\nResults saved to {filename}")
    
    def print_summary(self):
        print("\n" + "="*90)
        print("SUMMARY - GRADIENT BOOSTING CLASSIFIER")
        print("="*90)
        print(f"{'Clauses':<10} {'Examples':<10} {'Features':<10} {'Test Acc':<12} {'Test F1':<12} {'Precision':<12} {'Recall':<12}")
        print("-"*90)
        
        for result in self.results:
            print(f"{result['clauses']:<10} {result['examples']:<10} "
                  f"{result['n_features']:<10} "
                  f"{result['test_accuracy']:<12.4f} "
                  f"{result['test_f1_score']:<12.4f} "
                  f"{result['test_precision']:<12.4f} "
                  f"{result['test_recall']:<12.4f}")
        
        print("="*90)
        
        # Calculate and display averages
        avg_acc = np.mean([r['test_accuracy'] for r in self.results])
        avg_f1 = np.mean([r['test_f1_score'] for r in self.results])
        print(f"\n{'Average':<10} {'':<10} {'':<10} {avg_acc:<12.4f} {avg_f1:<12.4f}")
        print("="*90)
        
        print("\n" + "="*80)
        print("BEST HYPERPARAMETERS FOR EACH CONFIGURATION")
        print("="*80)
        
        for result in self.results:
            print(f"\n{result['clauses']} clauses, {result['examples']} examples:")
            print("-"*80)
            
            params_str = ", ".join([f"{k}={v}" for k, v in sorted(result['best_params'].items())])
            print(f"  Parameters: {params_str}")
            
            print(f"  Performance: Acc={result['test_accuracy']:.4f}, "
                  f"F1={result['test_f1_score']:.4f}, "
                  f"Prec={result['test_precision']:.4f}, "
                  f"Rec={result['test_recall']:.4f}")


def main():
    print("\nEXPERIMENT 5: GRADIENT BOOSTING CLASSIFIER")
    print("CNF Boolean Formula Classification")
    print("="*60)
    
    experiment = GradientBoostingCNFExperiment(data_dir='./all_data')
    experiment.run_all_experiments()
    
    print("\nEXPERIMENT COMPLETED")
    print("="*60)


if __name__ == "__main__":
    main()


EXPERIMENT 5: GRADIENT BOOSTING CLASSIFIER
CNF Boolean Formula Classification

GRADIENT BOOSTING CLASSIFIER - CNF CLASSIFICATION
Data directory: c:\utd\sem1\machinelearning\project2\project2_data\all_data
Optimized for speed (<10 minutes target)

Scanning for available datasets...
------------------------------------------------------------
✓ Found: 300 clauses, 100 examples
✓ Found: 300 clauses, 1000 examples
✓ Found: 300 clauses, 5000 examples
✓ Found: 500 clauses, 100 examples
✓ Found: 500 clauses, 1000 examples
✓ Found: 500 clauses, 5000 examples
✓ Found: 1000 clauses, 100 examples
✓ Found: 1000 clauses, 1000 examples
✓ Found: 1000 clauses, 5000 examples
✓ Found: 1500 clauses, 100 examples
✓ Found: 1500 clauses, 1000 examples
✓ Found: 1500 clauses, 5000 examples
✓ Found: 1800 clauses, 100 examples
✓ Found: 1800 clauses, 1000 examples
✓ Found: 1800 clauses, 5000 examples
------------------------------------------------------------
Total available datasets: 15/15

Processing 15 data

In [9]:
import numpy as np
import pandas as pd
import itertools
import json
import time
import gc

from sklearn.datasets import fetch_openml
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score


class MNISTClassifierExperiment:
    """Evaluate Decision Tree, Bagging, Random Forest, and Gradient Boosting on MNIST with param grids."""

    def __init__(self):
        self.results = []
        self.X_train = None
        self.X_test = None
        self.y_train = None
        self.y_test = None

    # -----------------------------------------------------
    # Data Loading
    # -----------------------------------------------------
    def load_and_preprocess_data(self):
        print("=" * 70)
        print("LOADING AND PREPROCESSING MNIST DATASET")
        print("=" * 70)

        start_time = time.time()

        X, y = fetch_openml("mnist_784", version=1, return_X_y=True, parser="auto")
        X = np.array(X, dtype=np.float32) / 255.0
        y = np.array(y)

        self.X_train, self.X_test = X[:60000], X[60000:]
        self.y_train, self.y_test = y[:60000], y[60000:]

        del X, y
        gc.collect()

        print(f"Dataset loaded in {time.time() - start_time:.1f}s")
        print(f"Training set: {self.X_train.shape}")
        print(f"Test set: {self.X_test.shape}")

    # -----------------------------------------------------
    # Helper to loop through parameter grids
    # -----------------------------------------------------
    def generate_param_combinations(self, grid_dict):
        keys = grid_dict.keys()
        values = grid_dict.values()
        for combo in itertools.product(*values):
            yield dict(zip(keys, combo))

    # -----------------------------------------------------
    # Experiment 1: Decision Tree
    # -----------------------------------------------------
    def experiment_1_decision_tree(self):
        print("\n" + "=" * 70)
        print("EXPERIMENT 1: DECISION TREE CLASSIFIER")
        print("=" * 70)

        start_time = time.time()
        grid = {
            "criterion": ["gini", "entropy"],
            "max_depth": [10, 20, 30],
            "min_samples_split": [5, 10],
            "min_samples_leaf": [2, 4],
            "max_features": ["sqrt", "log2"],
        }

        best_result = None
        for params in self.generate_param_combinations(grid):
            print(f"Testing: {params}")
            model = DecisionTreeClassifier(**params, random_state=42)
            model.fit(self.X_train, self.y_train)
            acc = accuracy_score(self.y_test, model.predict(self.X_test))
            print(f" → Accuracy: {acc:.4f}")
            if not best_result or acc > best_result["test_accuracy"]:
                best_result = {
                    "classifier": "Decision Tree",
                    "best_params": params,
                    "test_accuracy": float(acc),
                    "training_time": time.time() - start_time,
                }

        print(f"\nBest Decision Tree Accuracy: {best_result['test_accuracy']:.4f}")
        self.results.append(best_result)
        gc.collect()
        return best_result

    # -----------------------------------------------------
    # Experiment 2: Bagging Classifier
    # -----------------------------------------------------
    def experiment_2_bagging(self):
        print("\n" + "=" * 70)
        print("EXPERIMENT 2: BAGGING CLASSIFIER")
        print("=" * 70)

        start_time = time.time()
        bagging_grid = {
            "n_estimators": [25],
            "max_samples": [ 0.9],
            "max_features": [ 0.9],
        }
        base_tree_grid = {
            "criterion": ["gini"],
            "max_depth": [10, 25],
            "min_samples_split": [5, 10],
        }

        best_result = None
        for tree_params in self.generate_param_combinations(base_tree_grid):
            for bag_params in self.generate_param_combinations(bagging_grid):
                print(f"Testing: base={tree_params}, bag={bag_params}")
                base = DecisionTreeClassifier(**tree_params, random_state=42)
                bag = BaggingClassifier(
                    estimator=base, random_state=42, n_jobs=1, **bag_params
                )
                bag.fit(self.X_train, self.y_train)
                acc = accuracy_score(self.y_test, bag.predict(self.X_test))
                print(f" → Accuracy: {acc:.4f}")
                if not best_result or acc > best_result["test_accuracy"]:
                    best_result = {
                        "classifier": "Bagging",
                        "best_params": {**tree_params, **bag_params},
                        "test_accuracy": float(acc),
                        "training_time": time.time() - start_time,
                    }

        print(f"\nBest Bagging Accuracy: {best_result['test_accuracy']:.4f}")
        self.results.append(best_result)
        gc.collect()
        return best_result

    # -----------------------------------------------------
    # Experiment 3: Random Forest
    # -----------------------------------------------------
    def experiment_3_random_forest(self):
        print("\n" + "=" * 70)
        print("EXPERIMENT 3: RANDOM FOREST CLASSIFIER")
        print("=" * 70)

        start_time = time.time()
        grid = {
            "n_estimators": [50],
            "criterion": ["gini", "entropy"],
            "max_depth": [20, 30],
            "min_samples_split": [5, 10],
            "min_samples_leaf": [1, 2],
            "max_features": ["sqrt", "log2"],
        }

        best_result = None
        for params in self.generate_param_combinations(grid):
            print(f"Testing: {params}")
            rf = RandomForestClassifier(**params, n_jobs=1, random_state=42)
            rf.fit(self.X_train, self.y_train)
            acc = accuracy_score(self.y_test, rf.predict(self.X_test))
            print(f" → Accuracy: {acc:.4f}")
            if not best_result or acc > best_result["test_accuracy"]:
                best_result = {
                    "classifier": "Random Forest",
                    "best_params": params,
                    "test_accuracy": float(acc),
                    "training_time": time.time() - start_time,
                }

        print(f"\nBest Random Forest Accuracy: {best_result['test_accuracy']:.4f}")
        self.results.append(best_result)
        gc.collect()
        return best_result

    # -----------------------------------------------------
    # Experiment 4: Gradient Boosting
    # -----------------------------------------------------
    def experiment_4_gradient_boosting(self):
        print("\n" + "=" * 70)
        print("EXPERIMENT 4: GRADIENT BOOSTING CLASSIFIER")
        print("=" * 70)

        start_time = time.time()
        grid = {
            "n_estimators": [50],
            "learning_rate": [0.01, 0.1],
            "max_depth": [5, 7],
            "min_samples_split": [5, 10],
            "min_samples_leaf": [2, 4],
            "subsample": [0.8, 0.9],
            "max_features": ["sqrt"],
        }

        best_result = None
        for params in self.generate_param_combinations(grid):
            print(f"Testing: {params}")
            gb = GradientBoostingClassifier(**params, random_state=42)
            gb.fit(self.X_train, self.y_train)
            acc = accuracy_score(self.y_test, gb.predict(self.X_test))
            print(f" → Accuracy: {acc:.4f}")
            if not best_result or acc > best_result["test_accuracy"]:
                best_result = {
                    "classifier": "Gradient Boosting",
                    "best_params": params,
                    "test_accuracy": float(acc),
                    "training_time": time.time() - start_time,
                }

        print(f"\nBest Gradient Boosting Accuracy: {best_result['test_accuracy']:.4f}")
        self.results.append(best_result)
        gc.collect()
        return best_result

    # -----------------------------------------------------
    # Run All Experiments
    # -----------------------------------------------------
    def run_all_experiments(self):
        total_start = time.time()
        self.load_and_preprocess_data()

        print("\nStarting experiments...")
        self.experiment_1_decision_tree()
        self.experiment_2_bagging()
        self.experiment_3_random_forest()
        self.experiment_4_gradient_boosting()

        total_time = time.time() - total_start
        self.print_final_summary(total_time)
        self.save_results()

    # -----------------------------------------------------
    # Summary
    # -----------------------------------------------------
    def print_final_summary(self, total_time):
        print("\n" + "=" * 80)
        print("FINAL SUMMARY - ALL CLASSIFIERS ON MNIST")
        print("=" * 80)
        print(f"\n{'Classifier':<25} {'Test Accuracy':<20} {'Time (min)':<15}")
        print("-" * 80)

        for r in self.results:
            print(
                f"{r['classifier']:<25} "
                f"{r['test_accuracy']:<20.4f} "
                f"{r['training_time']/60:<15.1f}"
            )

        print("=" * 80)
        best = max(self.results, key=lambda x: x["test_accuracy"])
        print(f"\nBest Classifier: {best['classifier']} ({best['test_accuracy']:.4f})")
        print(f"Total runtime: {total_time/60:.1f} minutes")

    # -----------------------------------------------------
    # Save Results
    # -----------------------------------------------------
    def save_results(self):
        with open("mnist_results.json", "w") as f:
            json.dump(self.results, f, indent=2)
        print("\nResults saved to mnist_results.json")


# -----------------------------------------------------
# MAIN
# -----------------------------------------------------
def main():
    experiment = MNISTClassifierExperiment()
    experiment.run_all_experiments()
    print("\nALL EXPERIMENTS COMPLETED SUCCESSFULLY")


if __name__ == "__main__":
    main()


LOADING AND PREPROCESSING MNIST DATASET
Dataset loaded in 5.0s
Training set: (60000, 784)
Test set: (10000, 784)

Starting experiments...

EXPERIMENT 1: DECISION TREE CLASSIFIER
Testing: {'criterion': 'gini', 'max_depth': 10, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': 'sqrt'}
 → Accuracy: 0.7939
Testing: {'criterion': 'gini', 'max_depth': 10, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': 'log2'}
 → Accuracy: 0.7434
Testing: {'criterion': 'gini', 'max_depth': 10, 'min_samples_split': 5, 'min_samples_leaf': 4, 'max_features': 'sqrt'}
 → Accuracy: 0.7949
Testing: {'criterion': 'gini', 'max_depth': 10, 'min_samples_split': 5, 'min_samples_leaf': 4, 'max_features': 'log2'}
 → Accuracy: 0.7497
Testing: {'criterion': 'gini', 'max_depth': 10, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_features': 'sqrt'}
 → Accuracy: 0.7937
Testing: {'criterion': 'gini', 'max_depth': 10, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_features': 'log2'}
 → Acc